# Comics Rx
## [A comic book recommendation system](https://github.com/MangrobanGit/comics_rx)
<img src="https://images.unsplash.com/photo-1514329926535-7f6dbfbfb114?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&auto=format&fit=crop&w=2850&q=80" width="400" align='left'>

---

# 6 - ALS Model - Reduced Data, with Grid Search

This time, as explored in the EDA NB, let's consider removing customers who we feel have too few or too many purchases to influence the model in the intended way.

Examples:
- Too few - Customers who have only bought 1 comic (series).
- Too many - Customers with > 1000 series (for example, think all eBay customers are rolled into one account number).

# Libraries

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 1 #would be where you need to specify the files
%aimport comic_recs

import pandas as pd # dataframes
import os

# Data storage
from sqlalchemy import create_engine # SQL helper
import psycopg2 as psql #PostgreSQL DBs

# import necessary libraries
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
# from pyspark.sql.types import (StructType, StructField, IntegerType
#                                ,FloatType, LongType, StringType)
from pyspark.sql.types import *

import pyspark.sql.functions as F
from pyspark.sql.functions import col, explode, lit, isnan, when, count
from pyspark.ml.recommendation import ALS, ALSModel
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, TrainValidationSplit
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Custom
import data_fcns as dfc
import keys  # Custom keys lib
import comic_recs as cr

# Data storage
from sqlalchemy import create_engine # SQL helper
import psycopg2 as psql #PostgreSQL DBs

import time
import itertools

import numpy as np

In [2]:
# instantiate SparkSession object
spark = pyspark.sql.SparkSession.builder.master("local[*]").getOrCreate()
# spark = SparkSession.builder.master("local").getOrCreate()

## Import Data

We've previously set aside the dataset into a `json` file.

In [3]:
# We have previously created a version of the transactions table and filtered it down.
sold = spark.read.json('raw_data/als_input_filtered.json')

In [4]:
# Persist the data
sold.persist()

DataFrame[account_id: bigint, bought: bigint, comic_id: bigint]

### ALS Model

Let's start with  train/test split.

In [5]:
# Split data into training and test set
(train, test) = sold.randomSplit([.8, .2], seed=41916)

Make sure shapes make sense.

In [6]:
print(train.count(), len(train.columns))

49402 3


In [7]:
print(test.count(), len(test.columns))

12469 3


In [8]:
# Evaluate the model by computing the RMSE on the test data
eval_reg = RegressionEvaluator(metricName="rmse"
                               , labelCol="bought"
                               , predictionCol="prediction")

### Round 1

In [62]:
# hyper-param config
num_iterations = [15, 20, 30]
ranks = [5, 10, 15, 20]
reg_params = [0.01, 0.1, 0.5]
alphas = [1, 30, 40]

In [63]:
# grid search and select best model
start_time = time.time()
final_model = cr.train_ALS(train, test, eval_reg, num_iterations, reg_params, ranks, alphas)

print ('Total Runtime: {:.2f} seconds'.format(time.time() - start_time))

15 iterations,5 latent factors, regularization=0.01, and alpha @ 1 : validation RMSE is 0.8525
15 iterations,5 latent factors, regularization=0.01, and alpha @ 30 : validation RMSE is 0.5274
15 iterations,5 latent factors, regularization=0.01, and alpha @ 40 : validation RMSE is 0.4933
15 iterations,10 latent factors, regularization=0.01, and alpha @ 1 : validation RMSE is 0.8298
15 iterations,10 latent factors, regularization=0.01, and alpha @ 30 : validation RMSE is 0.5232
15 iterations,10 latent factors, regularization=0.01, and alpha @ 40 : validation RMSE is 0.4968
15 iterations,15 latent factors, regularization=0.01, and alpha @ 1 : validation RMSE is 0.8189
15 iterations,15 latent factors, regularization=0.01, and alpha @ 30 : validation RMSE is 0.5319
15 iterations,15 latent factors, regularization=0.01, and alpha @ 40 : validation RMSE is 0.5099
15 iterations,20 latent factors, regularization=0.01, and alpha @ 1 : validation RMSE is 0.8118
15 iterations,20 latent factors, regu

Py4JJavaError: An error occurred while calling o11560.fit.
: org.apache.spark.SparkException: Job 4289 cancelled because SparkContext was shut down
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$cleanUpAfterSchedulerStop$1.apply(DAGScheduler.scala:932)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$cleanUpAfterSchedulerStop$1.apply(DAGScheduler.scala:930)
	at scala.collection.mutable.HashSet.foreach(HashSet.scala:78)
	at org.apache.spark.scheduler.DAGScheduler.cleanUpAfterSchedulerStop(DAGScheduler.scala:930)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onStop(DAGScheduler.scala:2128)
	at org.apache.spark.util.EventLoop.stop(EventLoop.scala:84)
	at org.apache.spark.scheduler.DAGScheduler.stop(DAGScheduler.scala:2041)
	at org.apache.spark.SparkContext$$anonfun$stop$6.apply$mcV$sp(SparkContext.scala:1949)
	at org.apache.spark.util.Utils$.tryLogNonFatalError(Utils.scala:1340)
	at org.apache.spark.SparkContext.stop(SparkContext.scala:1948)
	at org.apache.spark.Sp

Crashed, but on inspection appears that `alpha` at 40 is best. Let's fix it at 40 and pare down some of the grid.

### Round 2

In [10]:
# hyper-param config
num_iterations = [15, 20]
ranks = [5, 10]
reg_params = [0.1, 0.5]
alphas = [40]

In [11]:
# grid search and select best model
start_time = time.time()
final_model = cr.train_ALS(train, test, eval_reg, num_iterations, reg_params, ranks, alphas)

print ('Total Runtime: {:.2f} seconds'.format(time.time() - start_time))

15 iterations, 5 latent factors, regularization=0.1, and alpha @ 40 : validation RMSE is 0.4948
15 iterations, 10 latent factors, regularization=0.1, and alpha @ 40 : validation RMSE is 0.4968
15 iterations, 5 latent factors, regularization=0.5, and alpha @ 40 : validation RMSE is 0.4979
15 iterations, 10 latent factors, regularization=0.5, and alpha @ 40 : validation RMSE is 0.4962
20 iterations, 5 latent factors, regularization=0.1, and alpha @ 40 : validation RMSE is 0.4944
20 iterations, 10 latent factors, regularization=0.1, and alpha @ 40 : validation RMSE is 0.4951
20 iterations, 5 latent factors, regularization=0.5, and alpha @ 40 : validation RMSE is 0.4969
20 iterations, 10 latent factors, regularization=0.5, and alpha @ 40 : validation RMSE is 0.4940
Total Runtime: 103.86 seconds


### Round 3

In [11]:
# hyper-param config
num_iterations = [20,30]
ranks = [5,10]
reg_params = [0.1]
alphas = [40]

In [12]:
# grid search and select best model
start_time = time.time()
final_model = cr.train_ALS(train, test, eval_reg, num_iterations, reg_params, ranks, alphas)

print ('Total Runtime: {:.2f} seconds'.format(time.time() - start_time))

20 iterations, 5 latent factors, regularization=0.1, and alpha @ 40 : validation RMSE is 0.4822
20 iterations, 10 latent factors, regularization=0.1, and alpha @ 40 : validation RMSE is 0.4767


Py4JJavaError: An error occurred while calling o276.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 3732.0 failed 1 times, most recent failure: Lost task 1.0 in stage 3732.0 (TID 3596, localhost, executor driver): java.lang.StackOverflowError
	at java.io.ObjectInputStream.readSerialData(ObjectInputStream.java:2211)
	at java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2069)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1573)
	at java.io.ObjectInputStream.readObject(ObjectInputStream.java:431)
	at scala.collection.immutable.List$SerializationProxy.readObject(List.scala:490)
	at sun.reflect.GeneratedMethodAccessor10.invoke(Unknown Source)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at java.io.ObjectStreamClass.invokeReadObject(ObjectStreamClass.java:1170)
	at java.io.ObjectInputStream.readSerialData(ObjectInputS

### Round 4

It seems none of my assets can support going beyond 30 iterations. Or it's something else. It's probably diminishing returns to try to pursue it any further, just select best model from the parameters I *have* been able to get to run.

In [11]:
# hyper-param config
num_iterations = [25]
ranks = [5,10,15,20]
reg_params = [0.1]
alphas = [40]

In [12]:
# grid search and select best model
start_time = time.time()
final_model = cr.train_ALS(train, test, eval_reg, num_iterations, reg_params, ranks, alphas)

print ('Total Runtime: {:.2f} seconds'.format(time.time() - start_time))

25 iterations, 5 latent factors, regularization=0.1, and alpha @ 40 : validation error is 0.4818
25 iterations, 10 latent factors, regularization=0.1, and alpha @ 40 : validation error is 0.4756
25 iterations, 15 latent factors, regularization=0.1, and alpha @ 40 : validation error is 0.4862
25 iterations, 20 latent factors, regularization=0.1, and alpha @ 40 : validation error is 0.5062
Total Runtime: 55.42 seconds


**OK**. Let's call it good. 

## Results 
Looks like the best parameters we could find are:
- `maxIter` = 25
- `rank` = 10
- `regParam` = 0.1 (default)
- `alpha` = 40

Let's create fit our candidate model.

## Candidate Model

In [9]:
# Create ALS instance and fit model
als = ALS(maxIter=25,
          rank=10,
          userCol='account_id',
          itemCol='comic_id',
          ratingCol='bought',
          implicitPrefs=True,
          regParam=0.1,
          alpha=40,
          coldStartStrategy='nan', # In the final we want to find our nan's
          seed=41916)
model_use = als.fit(train)

### Get Top N recommendations for Single User

Let's make a reference list of `account_id`'s, for testing purposes.

In [11]:
n_to_test = 2

users = (sold.select(als.getUserCol())
                          .sample(False
                                  ,n_to_test/sold.count()
                                  )
        )
users.persist()
users.show(2)

+----------+
|account_id|
+----------+
|       192|
|       569|
+----------+



We developed and wrote the functionality out to a function in `comic_recs.py`

###  Testing function!

- Pass the function to a pandas dataframe. 
- Function will ask for an account_id.
- Will return top n, n defined in parameters.

In [12]:
top_n_df = cr.get_top_n_new_recs(spark=spark, model=model_use, topn=5)
top_n_df

192


Py4JJavaError: An error occurred while calling o210.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 1658.0 failed 1 times, most recent failure: Lost task 0.0 in stage 1658.0 (TID 1069, localhost, executor driver): java.lang.StackOverflowError
	at java.lang.ReflectiveOperationException.<init>(ReflectiveOperationException.java:89)
	at java.lang.reflect.InvocationTargetException.<init>(InvocationTargetException.java:72)
	at sun.reflect.GeneratedMethodAccessor10.invoke(Unknown Source)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at java.io.ObjectStreamClass.invokeReadObject(ObjectStreamClass.java:1170)
	at java.io.ObjectInputStream.readSerialData(ObjectInputStream.java:2178)
	at java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2069)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1573)
	at java.io.Object

In [13]:
top_n_df = cr.get_top_n_new_recs(spark=spark, model=model_use, topn=5)
top_n_df

148


,comic_title
1,Head Lopper (Image)
2,Southern Bastards (Image)
3,Dark Horse Presents (Dark Horse)
4,Rocketeer Adventures 2 (IDW)
5,Fade Out (Image)


In [21]:
top_n_df = cr.get_top_n_new_recs(spark=spark, model=model_use, topn=10)
top_n_df

161


,comic_title
1,Aliens Dead Orbit (Dark Horse)
2,Beef (Image)
3,Vinegar Teeth (Dark Horse)
4,Leviathan (Image)
5,Shaolin Cowboy Wholl Stop th (Dark Horse)
6,Head Lopper (Image)
7,Crossed Badlands (Avatar)
8,Punisher (Marvel)
9,Godzilla Oblivion (IDW)
10,Nova (Marvel)


## Conclusions
- Seems realistic? Only three tests, but the results seem 'individualized' in the sense that there is no overlap between the sets (albeit small samples).

## Save the Model!

In [26]:
model_use.save('als_use')

## Retrieving Saved Model

In [27]:
comic_rec_model = ALSModel.load('als_use')

In [28]:
top_n_df = cr.get_top_n_new_recs(spark=spark, model=comic_rec_model, topn=10)
top_n_df

161


,comic_title
1,Aliens Dead Orbit (Dark Horse)
2,Beef (Image)
3,Vinegar Teeth (Dark Horse)
4,Leviathan (Image)
5,Shaolin Cowboy Wholl Stop th (Dark Horse)
6,Head Lopper (Image)
7,Crossed Badlands (Avatar)
8,Punisher (Marvel)
9,Godzilla Oblivion (IDW)
10,Nova (Marvel)
